In [3]:
from sentence_transformers import SentenceTransformer, util

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-distilbert-cos-v1')

In [2]:
query = "I just discovered the course. Can I still join it?"
docs = [""]

#Encode query and documents
query_emb = model.encode(query)
v = query_emb
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
#print(scores)

print ('Q1- What''s the first value of the resulting vector?')
print(query_emb[0])


Q1- Whats the first value of the resulting vector?
0.078222655


In [ ]:
query_emb

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
#filter
fitered_ml_docs = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':        
        fitered_ml_docs.append(doc)



In [7]:
fitered_ml_docs[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [8]:
#get embeddings_fitered_ml_docs
embeddings_fitered_ml_docs = []
for doc in fitered_ml_docs:
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings_fitered_ml_docs.append(model.encode(qa_text))

In [9]:
#convert with numpy
import numpy as np
x=[]
x = np.array(embeddings_fitered_ml_docs)


print ('Q2- What''s the shape of X? (X.shape)')
print(x.shape)

Q2- Whats the shape of X? (X.shape)
(375, 768)


In [10]:
scores = x.dot(v)
max = np.max(scores)
print ('Q3 - What''s the highest score in the results?')
print(max.round(2))


Q3 - Whats the shape of X? (X.shape)
0.65


In [19]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relevance_total)


def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [5]:
import pandas as pd
from tqdm.auto import tqdm

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
gt_question_embeddings = []


In [ ]:
#crate an array with all the question embeddings
for doc in tqdm(ground_truth):
    q = doc['question']
    q_v = model.encode(q)
    gt_question_embeddings.append(q_v)

In [11]:
import numpy as np
#convert to nnumpy
np_queston_emb=[]
np_queston_emb = np.array(gt_question_embeddings)

In [17]:
len(np_queston_emb)

1830

In [20]:
search_engine = VectorSearchEngine(documents=ground_truth, embeddings=np_queston_emb)

#results = search_engine.search(v, num_results=5)

In [23]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [22]:
search_vector = VectorSearchEngine(documents=gt_w_embeddings, embeddings=x)

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(q['question'], num_results=5)
    print(len(results))
    #relevance = [d['id'] == doc_id for d in results]
    #q['relevance'] = relevance
    #relevance_total.append(relevance)


#evaluate(ground_truth, search_function=search_engine.search)


  0%|          | 0/1830 [00:00<?, ?it/s]


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U35')) -> None

In [19]:
search_engine = VectorSearchEngine(documents=ground_truth, embeddings=x)
results = search_engine.search(v, num_results=5)
#print(ground_truth[0])
#len(ground_truth)

print(results[0])
len(results)

# search_engine = VectorSearchEngine(documents=ground_truth, embeddings=x)
# base_results = search_engine.search(v, num_results=5)

# base_results

# results = len(base_results)/len(ground_truth)
# print(len(ground_truth))
# print(len(base_results))
# print(results)

{'question': 'Can I still interact with instructors after missing a session?', 'course': 'machine-learning-zoomcamp', 'document': '5170565b'}


5